In [1]:
import os
from pathlib import Path
import sys

In [2]:
project_name = 'clpsych'
project_path = Path(os.getcwd()).parent

if sys.platform == "win32":
    data_path = 'D:\Dataset\{0}\dataset'.format(project_name)
    model_path = 'D:\Dataset\{0}\models'.format(project_name)
    src_path = '/Volumes/Dataset/{0}/src'.format(project_name)
    
elif sys.platform == 'darwin':
    data_path = '/Volumes/Dataset/{0}/dataset'.format(project_name)
    model_path = '/Volumes/Dataset/{0}/models'.format(project_name)
    src_path = '/Volumes/Dataset/{0}/src'.format(project_name)
    
else:
    data_path = Path(project_path, 'dataset')
    model_path = Path(project_path, 'models')
    src_path = Path(project_path, 'src')

utils_path = str(Path(project_path, 'utils'))
# including the project folder and the utils folder
if utils_path not in ''.join(sys.path):
    sys.path.extend([str(project_path), utils_path, str(src_path)])

print('project path = {0}'.format(project_path))
print('data path = {0}'.format(data_path))
print('model path = {0}'.format(model_path))
print('sys.path = {0}'.format(sys.path))

project path = /home/guerramarj/github/clpsych
data path = /home/guerramarj/github/clpsych/dataset
model path = /home/guerramarj/github/clpsych/models
sys.path = ['/cm/local/apps/cuda/libs/current/pynvml', '/home/guerramarj/python/Python-3.6.3', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python36.zip', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/lib-dynload', '', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages/Mako-1.0.7-py3.6.egg', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages/IPython/extensions', '/home/guerramarj/.ipython', '/home/guerramarj/github/clpsych', '/home/guerramarj/github/clpsych/utils', '/home/guerramarj/github/clpsych/src']


In [3]:
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from utils.datapath import data_path_scripts
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import precision_recall_fscore_support, classification_report
from numpy import interp 
from sklearn.metrics import confusion_matrix
import re
import itertools
from collections import Counter
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Dropout
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam, Adam
from keras.regularizers import l1_l2
from keras.models import Sequential
from keras.layers import Dense
from keras import backend
from keras import metrics
import tensorflow as tf
import _pickle as pickle 
import stanfordnlp
from keras.layers import LSTM
from keras.preprocessing.text import one_hot
from keras.datasets import imdb 
from utils.data_helpers import load_data_and_labels
from keras.preprocessing.text import text_to_word_sequence

%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 13]

# seed for numpy and sklearn
random_state = 7
np.random.seed(random_state)

Using TensorFlow backend.


In [4]:
import tensorflow
#confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())
assert 'GPU' in str(device_lib.list_local_devices())
print('Tensorflow recognizes GPUs')

# confirm Keras sees the GPU
from keras import backend
available_gpu = backend.tensorflow_backend._get_available_gpus()
assert len(available_gpu) > 0
print('number of available GPUs = {0}'.format(len(available_gpu)))
print('list of GPUs = {0}\n'.format(available_gpu))

Tensorflow recognizes GPUs
number of available GPUs = 2
list of GPUs = ['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1']



In [5]:
def resulting_confusion_matrix(y, y_pred, file_name, train_test, fig_number):
    
    print()
    print("\n{0} results\n".format(train_test))
    
    n_true = len(y[y == 0])
    n_false = len(y[y == 1])
    
    print('total data points = {0}'.format(y.shape[0]))
    print('true data points = {0} (% {1:.3})'.format(n_true, n_true / np.shape(y)[0]))
    print('false data points = {0} (% {1:.3})'.format(n_false, n_false / np.shape(y)[0]))
    
    fpr, tpr, _ = roc_curve(y, y_pred)
    roc_score = auc(fpr, tpr)
    r_auc_score = roc_auc_score(y, y_pred)
    
    print('roc score = {0}'.format(roc_score))
    print('roc auc score = {0}'.format(r_auc_score))
    
    print()    
    print("confusion matrix")

    target_names = ['Show', 'No-Show']
    print(classification_report(y, y_pred, target_names=target_names))

    # Compute confusion matrix
    cnf_matrix = confusion_matrix(y, y_pred)
    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    class_names = ['Show', 'No-Show']
    
    plt.figure(fig_number)
    plt.subplot(221)
    plot_confusion_matrix(cnf_matrix, classes=class_names, title='Confusion matrix, without normalization')
    print()
    
    plt.figure(fig_number)
    plt.subplot(222)
    # Plot non-normalized confusion matrix
    plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                          title='Normalized confusion matrix')
    print()
    
    plt.figure(fig_number)
    plt.subplot(223)
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_score)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    
    print()

In [6]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    return plt

In [7]:
def auc_metric(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    backend.get_session().run(tf.local_variables_initializer())
    return auc

In [8]:
%ls ../dataset

post_risklabel.csv*  preprocessed_corpus.pkl


In [9]:
person = 'wei'
if person == 'victor':
    data = pickle.load(Path(data_path, 'preprocessed_corpus.pkl').open('rb'))
elif person == 'wei':
    data = load_data_and_labels(file_path=data_path)

In [ ]:
vocabulary, vocabulary_inv = build_vocab(X_train)
vocabulary_size = len(vocabulary_inv)

In [ ]:
sentences_padded, sequence_length = pad_sentences(x_text)
X_train, y_train = build_input_data(X_train, y_train, vocabulary)
# print(X_train.shape)
X_test, y_test = build_input_data(X_test, y_test, vocabulary)
# all x and y for predicting
x, y_class = build_input_data(sentences_padded, y_class, vocabulary)

In [23]:
text = data['post_title'].str.cat(sep=' ')

In [24]:
text += data['post_body'].str.cat(sep=' ')

In [25]:
# integer encode the documents
words = set(text_to_word_sequence(text))
vocab_size = len(words)
encoded_docs = [one_hot(d, vocab_size) for d in data['post_title'] + data['post_body']]

In [27]:
top_words = 5000 
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=top_words)

In [ ]:
#reverse lookup
word_to_id = imdb.get_word_index()
word_to_id = {k:(v+0) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
id_to_word = {value:key for key,value in word_to_id.items()}
print(' '.join(id_to_word[id] for id in x_train[0] ))

In [ ]:
#one hot encode your documents
from numpy import array
from keras.preprocessing.text import one_hot
docs = ['Gut gemacht',
        'Gute arbeit',
        'Super idee',
        'Perfekt erledigt',
        'exzellent',
        'naja',
        'Schwache arbeit.',
        'Nicht gut',
        'Miese arbeit.',
        'Hätte es besser machen können.']
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

In [28]:
old_x_test = x_test

In [29]:
x_test = encoded_docs

In [30]:
# Truncate and pad the review sequences 
from keras.preprocessing import sequence 
max_review_length = 500 
x_train = sequence.pad_sequences(x_train, maxlen=max_review_length) 
x_test = sequence.pad_sequences(x_test, maxlen=max_review_length)

In [31]:
# Build the model 
embedding_vector_length = 32 
model = Sequential() 
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length)) 
model.add(LSTM(100)) 
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy']) 
print(model.summary()) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
# Build the model 
embedding_vector_length = 32 
model = Sequential() 
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length)) 
model.add(Flatten()) 
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy']) 
print(model.summary()) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 16001     
Total params: 176,001
Trainable params: 176,001
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
# model.fit(x_train, y_train, validation_data=(x_test, y_test), nb_epoch=3, batch_size=64) 
model.fit(x_train, y_train, epochs=3, batch_size=64) 

Epoch 1/3
25000/25000 [==============================] - 1s 54us/step - loss: 0.0314 - acc: 0.9974
Epoch 2/3
25000/25000 [==============================] - 1s 54us/step - loss: 0.0198 - acc: 0.9990
Epoch 3/3
25000/25000 [==============================] - 1s 54us/step - loss: 0.0128 - acc: 0.9996


In [ ]:
# %tensorboard --logdir=/home/guerramarj/logs

In [ ]:
#Evaluate the model
scores = model.evaluate(_test, y_test, verbose=0) 
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
#predict sentiment from reviews
bad = "this movie was terrible and bad"
good = "i really liked the movie and had fun"
for review in [good,bad]:
    tmp = []
    for word in review.split(" "):
        tmp.append(word_to_id[word])
    tmp_padded = sequence.pad_sequences([tmp], maxlen=max_review_length) 
    print("%s. Sentiment: %s" % (review,model.predict(array([tmp_padded][0]))[0][0]))

In [37]:
first, second = data[data['subreddit']  == 'SuicideWatch']['post_body'][0:2]

In [41]:
first, second = x_test[0], x_test[1]

In [45]:
model.predict_classes(x_test)

array([[1],
       [0],
       [1],
       ...,
       [0],
       [0],
       [1]], dtype=int32)